<u><b><p style="color:Red; text-align:center; background-color: yellow; font-size:300%">House Prices Estimation</u></b></p>

<b>Loading The data </b>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 500

In [3]:
df_train = pd.read_csv(r"C:\Kaggles\House Prices Predication\DataSet\train.csv")
df_test = pd.read_csv(r"C:\Kaggles\House Prices Predication\DataSet\test.csv")

print(df_train.shape, df_test.shape)

(1460, 81) (1459, 80)


<b>Data Cleaning and Feature Engineering</b>

In [4]:
df_train.drop('Id', axis=1, inplace=True)
Id = df_test["Id"]
df_test.drop("Id", axis=1, inplace=True)

df_test["SalePrice"] = np.nan

In [ ]:
combined_df = pd.concat([df_train, df_test])
combined_df.shape

In [ ]:
combined_df = pd.get_dummies(combined_df)

from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in combined_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    combined_df[column] = le.fit_transform(combined_df[column])
    label_encoders[column] = le

combined_df.shape

In [7]:
df_train = combined_df[combined_df["SalePrice"].notna()]
df_test = combined_df[combined_df["SalePrice"].isna()]
print(df_train.shape, df_test.shape)

(1460, 288) (1459, 288)


In [ ]:
df_test.drop("SalePrice", axis=1, inplace=True)

In [10]:
df_train = df_train.fillna(df_train.median()) 
df_test = df_test.fillna(df_test.median()) 

In [11]:
X = df_train.drop("SalePrice", axis=1)
Y = df_train.SalePrice

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.12, random_state=42)

In [14]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor

cv_split = ShuffleSplit(n_splits=10, test_size=0.12, random_state=False)
KFold_score_LR = cross_val_score(GradientBoostingRegressor(), X, Y, cv=cv_split)
print(KFold_score_LR)
print("\nAverage Score : ", np.mean(KFold_score_LR))

[0.87707075 0.87159676 0.84992868 0.93040218 0.88463906 0.88134191
 0.91210493 0.89530033 0.91210025 0.91874877]

Average Score :  0.8933233625760237


In [15]:
from xgboost import XGBRegressor

cv_split = ShuffleSplit(n_splits=10, test_size=0.12, random_state=False)
KFold_score_LR = cross_val_score(XGBRegressor(), X, Y, cv=cv_split)
print(KFold_score_LR)
print("\nAverage Score : ", np.mean(KFold_score_LR))

[0.80142709 0.8533268  0.7470411  0.88897366 0.88194527 0.88637388
 0.89076481 0.85483962 0.88024488 0.91897222]

Average Score :  0.8603909328261562


In [17]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline

def prediction(model):

    pipline = Pipeline([('model',model)])
    pipline.fit(X_train,Y_train)
    y_pred = pipline.predict(X_test)
    print(f"{model}")
    print(f"r2_score is {r2_score(Y_test, y_pred)}")
    print(f"mean_absolute_error is {mean_absolute_error(Y_test, y_pred)}")
    print(f"Root mean squared error is {np.sqrt(mean_squared_error(Y_test, y_pred))}")

In [18]:
best_model = GradientBoostingRegressor()
prediction(best_model)

GradientBoostingRegressor()
r2_score is 0.8924643448237254
mean_absolute_error is 16915.88816387009
Root mean squared error is 31486.45966895068


In [19]:
predictions = best_model.predict(df_test)

In [20]:
submission = pd.DataFrame({
    "Id" : Id,
    "SalePrice" : predictions
})

submission.to_csv(r"C:\Kaggles\House Prices Predication\Submissions\submission_2.csv", index = False)